In [1]:
import util
import numpy as np
import theano
import theano.tensor as T
import matplotlib.pyplot as plt

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
np.version.version

'1.20.3'

In [12]:
class Autoencoder:
    def __init__(self, D, M) :
        #represents a batch of training data
        self.X = T.matrix('X')

        #input -> hidden
        self.W = theano.shared(np.random.randn(D, M) * 2 / np.sqrt(M))
        self.b = theano.shared(np.zeros(M))

        #hidden -> output
        self.V = theano.shared(np.random.randn(M, D) * 2 / np.sqrt(D))
        self.c = theano.shared(np.zeros(D))

        #construct the reconstruction
        self.Z = T.nnet.relu(self.X.dot(self.W) + self.b)
        self.X_hat = T.nnet.sigmoid(self.Z.dot(self.V) + self.c)

        #compute the cost
        self.cost = T.sum(
            T.nnet.binary_crossentropy(
                output=self.X_hat,
                target=self.X,
            )
        )

        #define the updates
        params = [self.W, self.b, self.V, self.c]
        grads = T.grad(self.cost, params)

        #rmsprop
        decay = 0.9
        learning_date = 0.001

        #for rmsprop
        cache = [theano.shared(np.ones_like(p.get_value())) for p in params]
        new_cache = [decay * c + (1 - decay) *g*g for p, c, g in zip(params, cache, grads)]

        updates = [
            (c, new_c) for c, new_c in zip(cache, new_cache)
        ] + [
            (p, p- learning_date*g/T.sqrt(new_c + 1e-10)) for p, new_c, g in zip(params, new_cache, grads)
        ]

        #now define the callable functions
        self.train_op = theano.function(
            inputs = [self.X],
            outputs=self.cost,
            updates = updates
        )

        self.predict = theano.function(
            inputs = [self.X],
            outputs = self.X_hat
        )

    def fit(self, X, epochs = 30, batch_size = 64) :
        costs = []
        n_batches = len(X) // batch_size
        print(f'n_batches = {n_batches}')
        
        for i in range(epochs) :
            print(f'epoch : {i}')
            np.random.shuffle(X)
            for j in range(n_batches) :
                batch = X[j * batch_size : (j + 1) * batch_size]
                c = self.train_op(batch)
                c /= batch_size
                costs.append(c)
                if j % 100 == 0:
                    print(f'iter : {j}, cost : {c}')
        plt.plot(costs)
        plt.show()

In [8]:
X, Y = util.get_mnist()

Reading in and transforming data...


In [13]:
model = Autoencoder(784, 300)
model.fit(X)

ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: InplaceDimShuffle{x,x}(TensorConstant{0.5})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "C:\Users\Dhruv\Desktop\env\venv\lib\site-packages\theano\gof\opt.py", line 2034, in process_node
    replacements = lopt.transform(node)
  File "C:\Users\Dhruv\Desktop\env\venv\lib\site-packages\theano\tensor\opt.py", line 6515, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map,
  File "C:\Users\Dhruv\Desktop\env\venv\lib\site-packages\theano\gof\op.py", line 954, in make_thunk
    return self.make_c_thunk(node, storage_map, compute_map,
  File "C:\Users\Dhruv\Desktop\env\venv\lib\site-packages\theano\gof\op.py", line 857, in make_c_thunk
    outputs = cl.make_thunk(input_storage=node_input_storage,
  File "C:\Users\Dhruv\Desktop\env\venv\lib\site-packages\theano\gof\cc.py", line 1215, in make_thunk
    cthunk, m

ImportError: DLL load failed while importing m885ff006a95d626dac547a7bdfdb471bbf058622ece2b4435e42316c4012ea56: The specified module could not be found.